# **Машинное обучение ИБ-2024**

# **Домашнее задание 1.**
# Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [ ]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


Распакуем наши данные из архива.

In [ ]:
!unzip archice.zip

"unzip" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


Загрузим данные из csv файла в датафрейм.

In [ ]:
df = pd.read_csv('input_data.csv', delimiter=';')
df

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11358145,2021-12-31,6099000,4,9,3,65.0,0.0,56.041539,92.753133,0,0,660030.0,581436.0,24,857003.0
11358146,2021-12-31,2490000,1,10,2,56.9,0.0,55.169949,61.519210,0,0,454079.0,274414.0,74,1820769.0
11358147,2021-12-31,850000,2,2,2,37.0,5.0,55.946206,43.088179,0,0,606101.0,190983.0,52,958329.0
11358148,2021-12-31,4360000,5,5,1,36.0,9.0,61.256383,73.435919,0,0,628406.0,581702.0,86,2156710.0


Отобразим на карте координаты наших построек.

In [ ]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Добавляем точки на карту
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Формула гаверсинуса
    P = (math.sin(dlat / 2) ** 2 +
         math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2)
    Q = 2 * math.asin(math.sqrt(P))

    return Q * R

moscow_center = (55.755864, 37.617698)
spb_center = (59.938784, 30.314997)

df['is_Moscow'] = df.apply(lambda row: haversine_distance(row['geo_lat'], row['geo_lon'], moscow_center[0], moscow_center[1]) <= 20, axis=1)
df['is_Saint_Peterburg'] = df.apply(lambda row: haversine_distance(row['geo_lat'], row['geo_lon'], spb_center[0], spb_center[1]) <= 20, axis=1)

df

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0,False,False
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,False,False
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0,False,False
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,False,False
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11358145,2021-12-31,6099000,4,9,3,65.0,0.0,56.041539,92.753133,0,0,660030.0,581436.0,24,857003.0,False,False
11358146,2021-12-31,2490000,1,10,2,56.9,0.0,55.169949,61.519210,0,0,454079.0,274414.0,74,1820769.0,False,False
11358147,2021-12-31,850000,2,2,2,37.0,5.0,55.946206,43.088179,0,0,606101.0,190983.0,52,958329.0,False,False
11358148,2021-12-31,4360000,5,5,1,36.0,9.0,61.256383,73.435919,0,0,628406.0,581702.0,86,2156710.0,False,False


**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [ ]:
df = df.drop(columns=['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id'])
df

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,False,False
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,False,False
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,False,False
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,False,False
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,False,False
...,...,...,...,...,...,...,...,...,...,...
11358145,2021-12-31,6099000,4,9,3,65.0,0.0,0,False,False
11358146,2021-12-31,2490000,1,10,2,56.9,0.0,0,False,False
11358147,2021-12-31,850000,2,2,2,37.0,5.0,0,False,False
11358148,2021-12-31,4360000,5,5,1,36.0,9.0,0,False,False


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: building_type, is_Moscow, is_Saint_Peterburg

Числовые: date, price, level, levels, area, rooms, kitchen_area

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_col = ['building_type', 'is_Moscow', 'is_Saint_Peterburg']
enc = OneHotEncoder(drop='first')
enc_cat = enc.fit_transform(df[cat_col])
enc_df = pd.DataFrame(enc_cat.toarray(), columns=enc.get_feature_names_out(cat_col))
df = pd.concat([df.drop(columns=cat_col), enc_df], axis=1)
del enc_df
df

,date,price,level,levels,rooms,area,kitchen_area,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,is_Moscow_True,is_Saint_Peterburg_True
0,2021-01-01,2451300,15,31,1,30.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-01-01,10700000,4,13,3,85.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-01-01,2500000,2,3,1,30.0,9.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11358145,2021-12-31,6099000,4,9,3,65.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11358146,2021-12-31,2490000,1,10,2,56.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11358147,2021-12-31,850000,2,2,2,37.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11358148,2021-12-31,4360000,5,5,1,36.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [ ]:
from sklearn.preprocessing import StandardScaler

df['date'] = pd.to_datetime(df['date'])
df['days_since_first_observation'] = (df['date'] - df['date'].min()).dt.days
df['level_ratio'] = np.where(df['levels'] == 0, 0, df['level'] / df['levels'])
df = df.drop(columns=['date'])

num_col = ['level', 'levels', 'area', 'rooms', 'kitchen_area', 'days_since_first_observation', 'level_ratio']
scaler = StandardScaler()
df[num_col] = scaler.fit_transform(df[num_col])

df

,price,level,levels,rooms,area,kitchen_area,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,is_Moscow_True,is_Saint_Peterburg_True,days_since_first_observation,level_ratio
0,2451300,1.622769,2.665027,-0.621470,-0.840577,0.082486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.858355,-0.308125
1,1450000,-0.270043,-0.936859,-0.621470,-0.741051,0.267565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.858355,1.452742
2,10700000,-0.459324,0.171414,1.106234,1.175756,0.452644,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.858355,-0.909191
3,3100000,-0.648605,-0.936859,1.106234,1.065171,0.360105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.858355,0.088070
4,2500000,-0.837886,-1.213927,-0.621470,-0.851636,0.360105,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.858355,0.315515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11358145,6099000,-0.459324,-0.382722,1.106234,0.438523,0.082486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.752639,-0.442636
11358146,2490000,-1.027168,-0.244188,0.242382,0.139943,0.082486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.752639,-1.617770
11358147,850000,-0.837886,-1.352461,0.242382,-0.593604,0.236719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.752639,1.452742
11358148,4360000,-0.270043,-0.936859,-0.621470,-0.630466,0.360105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.752639,1.452742


**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [ ]:
import numpy as np
from sklearn.neighbors import BallTree

class KNNRegressor:
    def __init__(self, n_neighbors=5, metric="minkowski"):
        self.n_neighbors = n_neighbors
        self.metric = metric

    def fit(self, X, y):
        self.ball_tree = BallTree(X, metric=self.metric)
        self.y_train = np.array(y)

    def predict(self, X):
        X = np.array(X)
        predictions = [self._predict_single(x) for x in X]
        return np.array(predictions)

    def _predict_single(self, x):
        _, nearest_neighbor_ids = self.ball_tree.query([x], k=self.n_neighbors)
        nearest_neighbor_ids = nearest_neighbor_ids.flatten()
        nearest_neighbor_values = self.y_train[nearest_neighbor_ids]

        return np.mean(nearest_neighbor_values)


**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [ ]:
import numpy as np

class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None
        self.velocity = None
        self.grad_squared = None

    def _initialize_parameters(self, n_features):
        self.weights = np.zeros(n_features)
        self.bias = 0
        if self.optimization == 'Momentum':
            self.velocity = np.zeros(n_features)
        elif self.optimization == 'AdaGrad':
            self.grad_squared = np.zeros(n_features)

    def _compute_mse(self, X, y):
        y_pred = np.dot(X, self.weights) + self.bias
        mse = np.mean((y - y_pred) ** 2)
        return mse

    def _compute_gradients(self, X, y):
        y_pred = np.dot(X, self.weights) + self.bias
        error = y_pred - y
        n_samples = X.shape[0]

        dw = (2 / n_samples) * np.dot(X.T, error)
        db = (2 / n_samples) * np.sum(error)

        return dw, db

    def fit(self, X, y):
        self._initialize_parameters(X.shape[1])

        for _ in range(self.max_iter):
            dw, db = self._compute_gradients(X, y)

            if self.optimization == 'SGD':
                # Стохастический градиентный спуск
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db

            elif self.optimization == 'Momentum':
                # Градиентный спуск с импульсом
                self.velocity = self.decay_rate * self.velocity + self.learning_rate * dw
                self.weights -= self.velocity
                self.bias -= self.learning_rate * db

            elif self.optimization == 'AdaGrad':
                # Градиентный спуск с адаптивной скоростью обучения (AdaGrad)
                self.grad_squared += dw ** 2
                adjusted_learning_rate = self.learning_rate / (np.sqrt(self.grad_squared) + self.epsilon)
                self.weights -= adjusted_learning_rate * dw
                self.bias -= self.learning_rate * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias


# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression as SklearnLinearRegression

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def print_metrics(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse_value = rmse(y_true, y_pred)

    print(f"{model_name} Performance:")
    print(f"MSE: {mse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse_value:.4f}\n")


In [ ]:
y = df['price']
X = df.drop(columns=['price'])
del df

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Результаты линейной регрессии
lr_sklearn = SklearnLinearRegression()
lr_sklearn.fit(X_train, y_train)
y_pred_lr_sklearn = lr_sklearn.predict(X_test)
print_metrics(y_test, y_pred_lr_sklearn, "Sklearn Linear Regression")

lr_custom = LinearRegression(learning_rate=0.01, optimization='SGD', max_iter=100)
lr_custom.fit(X_train, y_train)
y_pred_lr_custom = lr_custom.predict(X_test)
print_metrics(y_test, y_pred_lr_custom, "Custom Linear Regression")



Sklearn Linear Regression Performance:
MSE: 186448090978221600.0000
MAE: 4485762.2339
RMSE: 431796353.5953

Custom Linear Regression Performance:
MSE: 186472214066279456.0000
MAE: 4432271.8753
RMSE: 431824286.1006



In [ ]:
# Результаты KNNRegressor
knn_sklearn = KNeighborsRegressor(n_neighbors=5)
knn_sklearn.fit(X_train, y_train)
y_pred_knn_sklearn = knn_sklearn.predict(X_test)
print_metrics(y_test, y_pred_knn_sklearn, "Sklearn KNN Regressor")

knn_custom = KNNRegressor(n_neighbors=5)
knn_custom.fit(X_train, y_train)
y_pred_knn_custom = knn_custom.predict(X_test)
print_metrics(y_test, y_pred_knn_custom, "Custom KNN Regressor")


Sklearn KNN Regressor Performance:
MSE: 186448731610643968.0000
MAE: 4369421.7067
RMSE: 431797095.4171

Custom KNN Regressor Performance:
MSE: 186528496405397792.0000
MAE: 5326657.3869
RMSE: 431889449.2870

